# 🤖 Importar librerías y cargar datos de inferencia
Este notebook importa las librerías principales y carga el archivo `ventas_2025_inferencia.csv` en un DataFrame llamado `inferencia_df`.

In [1]:
# Importar librerías principales para forecasting e inferencia
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import streamlit as st
import holidays

In [2]:
# Cargar el archivo de ventas de inferencia en un DataFrame
inferencia_df = pd.read_csv('../data/raw/inferencia/ventas_2025_inferencia.csv')
print('Primeras filas del DataFrame de inferencia:')
display(inferencia_df.head())

Primeras filas del DataFrame de inferencia:


,fecha,producto_id,nombre,categoria,subcategoria,precio_base,es_estrella,unidades_vendidas,precio_venta,ingresos,Amazon,Decathlon,Deporvillage
0,2025-10-25,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115,True,26.0,113.13,2941.38,89.51,113.43,104.78
1,2025-10-25,PROD_002,Adidas Ultraboost 23,Running,Zapatillas Running,135,True,27.0,141.89,3831.03,128.73,112.91,122.88
2,2025-10-25,PROD_003,Asics Gel Nimbus 25,Running,Zapatillas Running,85,False,5.0,85.79,428.95,84.28,74.51,85.57
3,2025-10-25,PROD_004,New Balance Fresh Foam X 1080v12,Running,Zapatillas Running,75,False,3.0,76.19,228.57,75.54,70.32,71.13
4,2025-10-25,PROD_005,Nike Dri-FIT Miler,Running,Ropa Running,35,False,3.0,35.48,106.44,33.84,31.32,34.41


In [5]:
# ...existing code...

# ============================================================================
# TRANSFORMACIÓN COMPLETA DE INFERENCIA_DF PARA FORECASTING
# ============================================================================

# 1. Convertir la columna 'fecha' a tipo datetime
inferencia_df['fecha'] = pd.to_datetime(inferencia_df['fecha'])

# 2. Crear variables temporales
inferencia_df['dia_semana'] = inferencia_df['fecha'].dt.dayofweek
inferencia_df['anio'] = inferencia_df['fecha'].dt.year
inferencia_df['mes'] = inferencia_df['fecha'].dt.month
inferencia_df['dia_mes'] = inferencia_df['fecha'].dt.day
inferencia_df['nombre_dia'] = inferencia_df['fecha'].dt.day_name()
inferencia_df['es_fin_semana'] = inferencia_df['dia_semana'].isin([5, 6]).astype(int)
inferencia_df['dia_anio'] = inferencia_df['fecha'].dt.dayofyear
inferencia_df['semana_anio'] = inferencia_df['fecha'].dt.isocalendar().week
inferencia_df['es_primer_dia_mes'] = (inferencia_df['dia_mes'] == 1).astype(int)
inferencia_df['es_ultimo_dia_mes'] = (inferencia_df['fecha'] == inferencia_df['fecha'] + pd.offsets.MonthEnd(0)).astype(int)
inferencia_df['trimestre'] = inferencia_df['fecha'].dt.quarter
inferencia_df['semestre'] = np.where(inferencia_df['mes'] <= 6, 1, 2)

# 3. Marcar festivos, Black Friday y Cyber Monday
es_holidays = holidays.country_holidays('ES', years=inferencia_df['anio'].unique())
inferencia_df['es_festivo'] = inferencia_df['fecha'].isin(es_holidays).astype(int)
inferencia_df['es_black_friday'] = ((inferencia_df['mes'] == 11) & (inferencia_df['dia_semana'] == 4) & (inferencia_df['dia_mes'] >= 23) & (inferencia_df['dia_mes'] <= 29)).astype(int)
inferencia_df['es_cyber_monday'] = ((inferencia_df['mes'] == 11) & (inferencia_df['dia_semana'] == 0) & (inferencia_df['dia_mes'] >= 26) & (inferencia_df['dia_mes'] <= 30)).astype(int)

# 4. Crear variables de lag y media móvil (por producto)
# IMPORTANTE: Si no tienes datos históricos previos, estos valores serán NaN
for lag in range(1, 8):
    inferencia_df[f'unidades_vendidas_lag{lag}'] = inferencia_df.groupby('producto_id')['unidades_vendidas'].shift(lag)

inferencia_df['unidades_vendidas_mm7'] = inferencia_df.groupby('producto_id')['unidades_vendidas'].rolling(window=7, min_periods=1).mean().reset_index(0, drop=True)

# 5. Variables de descuento y competencia
inferencia_df['descuento_porcentaje'] = ((inferencia_df['precio_base'] - inferencia_df['precio_venta']) / inferencia_df['precio_base']).replace([np.inf, -np.inf], 0).fillna(0)

# Si no tienes datos de competencia, crear columnas vacías
if 'precio_competencia' not in inferencia_df.columns:
    inferencia_df['precio_competencia'] = np.nan

inferencia_df['ratio_precio'] = inferencia_df['precio_venta'] / inferencia_df['precio_base']

# 6. Codificación one-hot para nombre, categoría y subcategoría
nombres = [
    'Adidas Own The Run Jacket', 'Adidas Ultraboost 23', 'Asics Gel Nimbus 25', 'Bowflex SelectTech 552',
    'Columbia Silver Ridge', 'Decathlon Bandas Elásticas Set', 'Domyos BM900', 'Domyos Kit Mancuernas 20kg',
    'Gaiam Premium Yoga Block', 'Liforme Yoga Pad', 'Lotuscrafts Yoga Bolster', 'Manduka PRO Yoga Mat',
    'Merrell Moab 2 GTX', 'New Balance Fresh Foam X 1080v12', 'Nike Air Zoom Pegasus 40', 'Nike Dri-FIT Miler',
    'Puma Velocity Nitro 2', 'Quechua MH500', 'Reebok Floatride Energy 5', 'Reebok Professional Deck',
    'Salomon Speedcross 5 GTX', 'Sveltus Kettlebell 12kg', 'The North Face Borealis', 'Trek Marlin 7'
]
for nombre in nombres:
    inferencia_df[f'nombre_h_{nombre}'] = (inferencia_df['nombre'] == nombre).astype(int)

categorias = ['Fitness', 'Outdoor', 'Running', 'Wellness']
for categoria in categorias:
    inferencia_df[f'categoria_h_{categoria}'] = (inferencia_df['categoria'] == categoria).astype(int)

subcategorias = [
    'Banco Gimnasio', 'Bandas Elásticas', 'Bicicleta Montaña', 'Bloque Yoga', 'Cojín Yoga',
    'Esterilla Fitness', 'Esterilla Yoga', 'Mancuernas Ajustables', 'Mochila Trekking', 'Pesa Rusa',
    'Pesas Casa', 'Rodillera Yoga', 'Ropa Montaña', 'Ropa Running', 'Zapatillas Running', 'Zapatillas Trail'
]
for subcategoria in subcategorias:
    inferencia_df[f'subcategoria_h_{subcategoria}'] = (inferencia_df['subcategoria'] == subcategoria).astype(int)

# 7. Variables hash originales (mantener como columnas adicionales)
inferencia_df['nombre_h'] = inferencia_df['nombre']
inferencia_df['categoria_h'] = inferencia_df['categoria']
inferencia_df['subcategoria_h'] = inferencia_df['subcategoria']

# 8. Eliminar registros de octubre y dejar solo noviembre
print(f'Registros antes de filtrar: {len(inferencia_df)}')
print(f'Meses únicos: {sorted(inferencia_df["mes"].unique())}')

inferencia_df = inferencia_df[inferencia_df['mes'] == 11].copy()

print(f'Registros después de filtrar (solo noviembre): {len(inferencia_df)}')
print(f'Meses únicos después del filtro: {sorted(inferencia_df["mes"].unique())}')

# 9. Verificar estructura final
print('\n📊 Estructura final de inferencia_df:')
print(f'Forma del DataFrame: {inferencia_df.shape}')
print(f'\nColumnas creadas: {inferencia_df.columns.tolist()}')
print(f'\nPrimeras filas:')
display(inferencia_df.head())

# 10. Verificar valores nulos en variables importantes
print('\n⚠️ Valores nulos por columna:')
nulos = inferencia_df.isnull().sum()
print(nulos[nulos > 0])

# 11. Guardar el dataframe transformado
ruta_salida = '../data/processed/inferencia_df_transformado.csv'
inferencia_df.to_csv(ruta_salida, index=False)
print(f'\n✅ Archivo guardado exitosamente en: {ruta_salida}')
print(f'Total de registros guardados: {len(inferencia_df)}')

Registros antes de filtrar: 720
Meses únicos: [np.int32(11)]
Registros después de filtrar (solo noviembre): 720
Meses únicos después del filtro: [np.int32(11)]

📊 Estructura final de inferencia_df:
Forma del DataFrame: (720, 86)

Columnas creadas: ['fecha', 'producto_id', 'nombre', 'categoria', 'subcategoria', 'precio_base', 'es_estrella', 'unidades_vendidas', 'precio_venta', 'ingresos', 'Amazon', 'Decathlon', 'Deporvillage', 'dia_semana', 'anio', 'mes', 'dia_mes', 'nombre_dia', 'es_fin_semana', 'dia_anio', 'semana_anio', 'es_primer_dia_mes', 'es_ultimo_dia_mes', 'trimestre', 'semestre', 'es_festivo', 'es_black_friday', 'es_cyber_monday', 'unidades_vendidas_lag1', 'unidades_vendidas_lag2', 'unidades_vendidas_lag3', 'unidades_vendidas_lag4', 'unidades_vendidas_lag5', 'unidades_vendidas_lag6', 'unidades_vendidas_lag7', 'unidades_vendidas_mm7', 'descuento_porcentaje', 'precio_competencia', 'ratio_precio', 'nombre_h_Adidas Own The Run Jacket', 'nombre_h_Adidas Ultraboost 23', 'nombre_h_Asi

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_38136\198727231.py:26: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  inferencia_df['es_festivo'] = inferencia_df['fecha'].isin(es_holidays).astype(int)


,fecha,producto_id,nombre,categoria,subcategoria,precio_base,es_estrella,unidades_vendidas,precio_venta,ingresos,...,subcategoria_h_Pesa Rusa,subcategoria_h_Pesas Casa,subcategoria_h_Rodillera Yoga,subcategoria_h_Ropa Montaña,subcategoria_h_Ropa Running,subcategoria_h_Zapatillas Running,subcategoria_h_Zapatillas Trail,nombre_h,categoria_h,subcategoria_h
168,2025-11-01,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115,True,NaN,115.00,NaN,...,0,0,0,0,0,1,0,Nike Air Zoom Pegasus 40,Running,Zapatillas Running
169,2025-11-01,PROD_002,Adidas Ultraboost 23,Running,Zapatillas Running,135,True,NaN,135.00,NaN,...,0,0,0,0,0,1,0,Adidas Ultraboost 23,Running,Zapatillas Running
170,2025-11-01,PROD_003,Asics Gel Nimbus 25,Running,Zapatillas Running,85,False,NaN,86.39,NaN,...,0,0,0,0,0,1,0,Asics Gel Nimbus 25,Running,Zapatillas Running
171,2025-11-01,PROD_004,New Balance Fresh Foam X 1080v12,Running,Zapatillas Running,75,False,NaN,74.09,NaN,...,0,0,0,0,0,1,0,New Balance Fresh Foam X 1080v12,Running,Zapatillas Running
172,2025-11-01,PROD_005,Nike Dri-FIT Miler,Running,Ropa Running,35,False,NaN,34.76,NaN,...,0,0,0,0,1,0,0,Nike Dri-FIT Miler,Running,Ropa Running



⚠️ Valores nulos por columna:
unidades_vendidas         720
ingresos                  720
unidades_vendidas_lag1    720
unidades_vendidas_lag2    720
unidades_vendidas_lag3    720
unidades_vendidas_lag4    720
unidades_vendidas_lag5    720
unidades_vendidas_lag6    720
unidades_vendidas_lag7    720
unidades_vendidas_mm7     720
precio_competencia        720
dtype: int64

✅ Archivo guardado exitosamente en: ../data/processed/inferencia_df_transformado.csv
Total de registros guardados: 720


In [7]:
inferencia_df.columns



Index(['fecha', 'producto_id', 'nombre', 'categoria', 'subcategoria',
       'precio_base', 'es_estrella', 'unidades_vendidas', 'precio_venta',
       'ingresos', 'Amazon', 'Decathlon', 'Deporvillage', 'dia_semana', 'anio',
       'mes', 'dia_mes', 'nombre_dia', 'es_fin_semana', 'dia_anio',
       'semana_anio', 'es_primer_dia_mes', 'es_ultimo_dia_mes', 'trimestre',
       'semestre', 'es_festivo', 'es_black_friday', 'es_cyber_monday',
       'unidades_vendidas_lag1', 'unidades_vendidas_lag2',
       'unidades_vendidas_lag3', 'unidades_vendidas_lag4',
       'unidades_vendidas_lag5', 'unidades_vendidas_lag6',
       'unidades_vendidas_lag7', 'unidades_vendidas_mm7',
       'descuento_porcentaje', 'precio_competencia', 'ratio_precio',
       'nombre_h_Adidas Own The Run Jacket', 'nombre_h_Adidas Ultraboost 23',
       'nombre_h_Asics Gel Nimbus 25', 'nombre_h_Bowflex SelectTech 552',
       'nombre_h_Columbia Silver Ridge',
       'nombre_h_Decathlon Bandas Elásticas Set', 'nombre_h_Dom